# Github commits dataset

In [4]:
import json
import requests
import pandas as pd
import numpy as np
import config

In [5]:
github_api = "https://api.github.com"
gh_session = requests.Session()
gh_session.auth = (config.GITHUB_USERNAME, config.GITHUB_TOKEN)

In [6]:
url = github_api + '/repos/processing/processing4/commits'
commits = gh_session.get(url = url)
commits_json = commits.json()

In [7]:
def commits_of_repo_github(repo, owner, api):
    commits = []
    next = True
    i = 1
    while next == True:
        url = api + '/repos/{}/{}/commits?page={}&per_page=100'.format(owner, repo, i)
        commit_pg = gh_session.get(url = url)
        commit_pg_list = [dict(item, **{'repo_name':'{}'.format(repo)}) for item in commit_pg.json()]    
        commit_pg_list = [dict(item, **{'owner':'{}'.format(owner)}) for item in commit_pg_list]
        commits = commits + commit_pg_list
        if 'Link' in commit_pg.headers:
            if 'rel="next"' not in commit_pg.headers['Link']:
                next = False
        i = i + 1
    return commits

In [8]:
commits = commits_of_repo_github("processing4", "processing", "https://api.github.com")
flat_commits = pd.json_normalize(commits)
# df = pd.DataFrame(commits)
# df.to_csv("processing-commits.csv")
df_flatcommits = pd.DataFrame(flat_commits)
df_flatcommits.to_csv("processing-commits-flat.csv")